# Calculate the percentage of anomaly
#### Jose Valles (jose.valles.leon@gmail.com)

### Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
import numpy as np
import calendar

sns.set()

from IPython.display import HTML

Import basin level code 2 and 3

In [2]:
BASIN_LEVEL3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',index_col="Codigo")
BASIN_LEVEL2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',index_col="Codigo")

In [3]:
def importmodelvariable(model_variable):
    df = pd.read_csv(f'../output_modelo/{model_variable}.csv')
    df = df.rename(columns={'-1': 'year','-1.1':'month'})
    df['date'] = pd.to_datetime(dict(year=df['year'],month=df['month'],day=1))
    df = df.set_index('date')
    df['days_in_month'] = df.index.days_in_month
    return df

def convertRunoff2Discharge(df_runoff):
    df_runoff_selected = df_runoff.drop(['year','month','days_in_month'],axis=1)
    df_discharge = pd.DataFrame(df_runoff_selected.values*1000*BASIN_LEVEL3.values,columns=df_runoff_selected.columns)
    df_discharge['days_in_month'] = df_runoff['days_in_month'].values
    df_discharge = df_discharge.loc[:, df_discharge.columns != 'days_in_month'].divide(df_discharge["days_in_month"]*24*3600, axis="index")
    df_discharge['date'] = df_runoff.index.values
    df_discharge = df_discharge.set_index('date')
    df_discharge['year'] = df_runoff['year'].values
    df_discharge['month'] = df_runoff['month'].values
    return df_discharge

In [4]:
hydrological_variable = ['Escorrentia_total','Escorrentia_sup','Escorrentia_sub','Pmedias','ETR','HumedadSuelo']

for hydro in hydrological_variable:
    if hydro == "Escorrentia_total":
        RUNOFF_total = importmodelvariable(hydro)
    elif hydro == "Pmedias":
        PRECIP = importmodelvariable(hydro)
    elif hydro == "ETR":
        ETR = importmodelvariable(hydro)
    elif hydro == "HumedadSuelo":
        SM = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sup':
        RUNOFF_sup = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sub':
        RUNOFF_sub = importmodelvariable(hydro)

In [5]:
HTML(RUNOFF_total.tail(6).to_html(index=False))

year,month,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,days_in_month
2023,6,21.162,0.92784,5.6434,8.3118,1.72440,1.8193,0.91421,0.082674,0.120180,0.005914,0.014234,0.025378,0.118210,0.163180,0.134830,0.42582,0.097249,0.043905,0.235070,0.049402,8.2056,0.12353,0.036598,0.021861,6.0157,0.042275,0.020247,0.049421,0.76946,1.01030,0.51685,1.80090,0.78328,1.33520,0.54676,1.5017,1.31440,1.70900,1.63300,1.49290,1.07700,0.32598,0.049116,14.855,0.51775,0.61622,5.9177,0.61944,0.80751,0.46071,0.49770,0.27177,0.003875,0.005875,0.193090,0.013620,2.508500e-06,0.05552,4.014000e-07,2.956400e-07,5.530500e-08,7.509700e-08,5.464700e-08,0.000001,2.833600e-07,1.042000e-07,8.305600e-08,6.920900e-08,8.685500e-08,7.830300e-08,5.524400e-08,5.574800e-08,7.879700e-08,8.609900e-08,1.074700e-07,1.032700e-07,1.027800e-07,1.007600e-07,2.437300e-06,0.000002,2.176800e-06,0.000002,1.547400e-06,1.182500e-06,2.526100e-07,3.927700e-07,8.694500e-08,9.344000e-08,8.860800e-08,1.750400e-07,9.270700e-07,1.209400e-06,6.996500e-07,2.183200e-07,3.939600e-07,3.848800e-07,2.652600e-07,0.000230,0.000299,0.000271,0.000219,0.000509,0.000261,0.001642,0.001417,0.001585,0.001270,0.000943,0.11833,0.002156,0.002008,0.097322,0.49809,0.001776,0.001624,0.012839,0.002647,0.001375,0.001367,0.001717,0.003052,0.52667,0.151100,0.006593,0.006729,1.0669,0.77345,0.007093,0.007626,1.0572,0.49872,0.011603,0.41569,0.013090,0.31684,0.18514,0.35536,0.92368,3.455,5.6755,16.873,8.3436,11.941,5.6478,6.661,10.132,16.982,24.190,22.587,18.623,7.9022,9.0478,2.5159,2.1372,6.7349,6.3159,2.0901,6.2032,18.346,6.0402,2.6252,1.5518,15.675,6.8072,9.3531,1.4083,7.9652,2.7065,3.9152,1.3156,0.77246,1.2763,0.6169,1.2052,0.43026,9.0753,0.49314,2.8592,3.3409,1.0833,0.22025,3.5503,0.62275,4.3332,4.1375,7.0631,0.099129,0.010248,1.4424,5.4713,12.976,6.9229,0.96088,0.011463,0.011353,0.010926,6.9333,9.6687,1.6771,0.016669,0.013329,0.015583,0.015101,0.016389,0.016535,0.016284,0.016138,0.014845,0.015672,0.014002,0.5351,0.010661,0.008476,0.013623,0.19054,0.008025,0.006207,0.007283,1.5383,0.28321,0.060438,5.7531,0.90454,0.006449,0.75595,0.008162,0.016973,0.031168,0.098837,0.58154,0.56877,0.30777,0.82115,0.89353,1.4802,1.1942,1.45700,1.50460,1.0498,0.68771,0.62188,8.9138,0.000008,11.398,0.045573,0.57194,0.37008,0.047916,0.000180,0.34434,0.000537,0.67846,13.8580,0.000019,0.97743,0.65711,0.56341,0.001935,0.002023,0.252120,0.001600,0.001486,0.000976,0.000723,1.00180,0.49450,0.26914,0.134540,0.008939,0.016168,0.000277,7.366400e-07,0.000045,0.000039,0.000013,2.706900e-07,3.829400e-07,0.055826,2.376500e-06,0.012721,0.10350,0.165070,0.046141,1.822000e-06,5.834100e-07,7.428700e-07,1.475600e-07,1.718400e-07,9.388600e-08,1.897400e-07,0.000050,0.000006,0.000148,5.205800e-08,1.666300e-07,0.000101,30
2023,7,20.771,3.11320,7.5701,9.5943,0.64076,1.3888,0.33970,0.030720,0.044655,0.002198,0.005289,0.009430,0.043925,0.060634,0.

### Convert from runoff to discharge

In [7]:
DISCHARGE = convertRunoff2Discharge(RUNOFF_total)

In [8]:
HTML(DISCHARGE.tail(6).to_html(index=True))

,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,year,month
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-06-01,6.858056,0.361543,1.770094,0.567588,0.276756,1.035288,0.853545,0.034384,0.002736,0.001043,0.000439,0.002369,0.009668,0.003525,0.015553,0.194018,0.015870,0.008029,0.078447,0.007490,0.702794,0.046324,0.006792,0.010568,1.288084,0.060183,0.005843,0.015139,0.218489,0.101732,0.001994,0.164666,0.322740,0.163809,0.359866,0.435678,0.487829,0.055384,0.090722,0.149751,0.360662,0.045149,0.031778,11.364763,0.069313,0.060861,3.698562,0.153665,0.087231,0.080518,0.151883,0.053578,0.000085,0.001312,0.183331,0.003379,1.258121e-08,0.066787,4.305139e-08,2.657566e-08,1.920312e-10,1.466014e-08,2.123053e-08,3.706466e-08,1.989642e-08,3.979861e-09,3.076148e-09,2.776904e-09,6.601248e-09,6.676297e-09,1.274534e-08,1.505540e-10,6.110416e-09,5.082233e-09,7.587581e-09,6.374691e-10,1.863681e-09,1.236176e-08,6.958341e-08,2.143472e-08,1.268120e-07,8.565903e-08,1.683514e-07,1.751852e-07,5.749996e-09,9.501034e-08,3.622708e-09,1.838519e-09,5.811481e-10,3.579136e-09,1.391320e-07,2.402936e-07,9.609390e-08,8.843981e-09,2.781431e-08,1.663062e-08,1.228056e-08,0.000005,0.000008,0.000017,0.000021,0.000166,2.016204e-06,0.000408,0.000036,0.000013,0.000007,0.000012,0.008720,0.000063,0.000126,0.004581,0.072446,0.000106,0.000017,0.000159,0.000063,0.000070,0.000016,0.000052,0.000040,0.144672,0.000816,0.000099,0.000018,0.356457,0.145619,0.000358,0.000074,0.289996,0.106979,0.000175,0.194694,0.000520,0.168077,0.008857,0.084316,0.689196,0.134628,0.208014,12.186056,0.312241,1.156324,0.557807,0.827485,2.736265,5.955493,0.550621,2.631664,5.683176,0.722539,2.988008,0.019413,0.199538,0.407940,1.376730,0.191915,0.447530,1.316495,0.163123,0.219779,0.421477,5.013339,3.873696,5.167299,0.111382,0.722154,0.476144,0.922912,0.547659,0.477125,1.519053,0.287030,0.502632,0.020417,9.222353,0.049466,0.669573,0.299031,0.422537,0.078600,0.104098,0.001442,0.618551,0.549113,3.013807,0.026006,0.007690,0.541456,2.528788,6.523043,3.290514,0.977933,0.000584,0.000526,0.001598,1.548758,1.316764,0.277576,0.004270,0.018333,0.000679,0.002732,0.004711,0.004816,0.007112,0.005367,0.000269,0.002273,0.002247,0.581343,0.009542,0.000952,0.002843,0.140332,0.002050,0.002153,0.004776,0.636804,0.124997,0.014690,0.026635,0.273944,0.000301,0.075537,0.006105,0.004531,0.005002,0.007664,0.130129,0.106644,0.209573,0.329157,0.165469,0.896572,0.528452,0.797078,0.922380,1.025904,0.444942,0.122601,0.055023,0.000010,2.106343,0.001266,0.607686,0.156056,0.007635,0.000019,0.115046,0.000036,0.214113,9.452525,0.000003,0.101061,0.211938,0.088250,0.000031,0.

### Pleasee select the runoff type for the analysis (RUNOFF_total, RUNOFF_sub, RUNOFF_sup)

In [9]:
# Select the runoff type 
RUNOFF = RUNOFF_total

### Select reference period from 'year_start' to 'year_end'

Select variable of interest

In [10]:
year_start = 1981
year_end = 2010
# Caudal
SELECTED_REF_DISCHARGE= DISCHARGE[(DISCHARGE['year'] >= year_start) & (DISCHARGE['year'] <= year_end)]
# Escorrentia
SELECTED_REF_RUNOFF = RUNOFF[(RUNOFF['year'] >= year_start) & (RUNOFF['year'] <= year_end)]
# Precip
SELECTED_REF_PRECIP = PRECIP[(PRECIP['year'] >= year_start) & (PRECIP['year'] <= year_end)]

## Make a water balance for the whole country (Uruguay)

#### Annual and Monthly Analysis

In [11]:
RUNOFF['avg_runoff'] = RUNOFF.iloc[:,2:-1].mean(axis=1)

In [12]:
sum_avg_runoff_year = RUNOFF.groupby(['year'])['avg_runoff'].sum()
sum_avg_runoff_year = sum_avg_runoff_year.to_frame()
sum_avg_runoff_month = RUNOFF.groupby(['year','month'])['avg_runoff'].sum()
sum_avg_runoff_month = sum_avg_runoff_month.to_frame()

In [13]:
display(sum_avg_runoff_year.tail(5))

,avg_runoff
year,
2019,629.945088
2020,169.752584
2021,317.016068
2022,222.449937
2023,135.701266


In [12]:
# sum_avg_runoff_month.to_clipboard()